In [1]:
using Rays

## Creating a scene

In [2]:
# Define the scene
scene = Rays.Scene()

# Define a camera
camera = Rays.Camera(; screen_res = [500, 500], intersection_data_variables = [:dim])
push!(scene, camera)

# Define a Menger sponge at the origin
menger_sponge = Rays.menger_sponge(zeros(Float32, 3), 1.0f0, 3)
push!(scene, menger_sponge)

## Creating a parameters object

In [3]:
julia_green = Float32[0.22, 0.596, 0.149]
julia_purple = Float32[0.584, 0.345, 0.698]
julia_red = Float32[0.796, 0.235, 0.2]
julia_colors = hcat(julia_green, julia_purple, julia_red)

mutable struct Params{F} <: Rays.Parameters{F}
    θ::F
    ϕ::F
    dθ::F
    dist::F
    colors::Matrix{F}
end

params = Params(Float32(π/4), Float32(3π/8), 0.0f0, 5.0f0, julia_colors)

Params{Float32}(0.7853982f0, 1.1780972f0, 0.0f0, 5.0f0, Float32[0.22 0.584 0.796; 0.596 0.345 0.235; 0.149 0.698 0.2])

## Creating parameter update functions

In [4]:
import SimpleDirectMediaLayer as SDL
using SimpleDirectMediaLayer.LibSDL2

"""
Actions:
- Change the horizontal rotation speed dθ (left/right keys)
- Change the vertical position of the camera given by ϕ (up/down)
"""
function affect_parameters_input!(params::Params{F}, event::SDL_Event, Δt::F)::Bool where {F}
    parameters_changed = false
    event_type = event.type

    if event_type == SDL.SDL_KEYDOWN
        scan_code = event.key.keysym.scancode 
        if scan_code == SDL.SDL_SCANCODE_RIGHT
            params.dθ += 2*Δt
            parameters_changed = true
        elseif scan_code == SDL.SDL_SCANCODE_LEFT
            params.dθ -= 2*Δt
            parameters_changed = true
        elseif scan_code == SDL.SDL_SCANCODE_UP
            params.ϕ = max(params.ϕ - Δt, 1e-3)
            parameters_changed = true
        elseif scan_code == SDL.SDL_SCANCODE_DOWN
            params.ϕ = min(params.ϕ + Δt, π-1e-3)
            parameters_changed = true
        end
    end
    return parameters_changed
end

function affect_parameters_time!(params::Params{F}, Δt::F)::Bool where {F}
    params.θ += params.dθ * Δt
    return true
end

affect_parameters_time! (generic function with 1 method)

## Creating a rendering function

In [5]:
function get_render(scene::Rays.Scene{F}, params::Rays.Parameters{F})::Array{F, 3} where {F <: AbstractFloat}

    cam = scene.cameras[:camera]

    # Adjust camera
    Rays.look_at!(cam, zeros(Float32,3), params.dist, params.θ, params.ϕ)

    # Get intersection data
    Rays.shape_view!(scene)

    # Get grayscale image
    Rays.cam_is_source!(cam, dropoff_curve = x -> clamp(4.0-x/1.4, 0.0, 1.0));

    # Apply color to the image
    Rays.get_color!(cam, :dim, params.colors)
    Rays.apply_color!(cam)

    return cam.canvas_color
end 

get_render (generic function with 1 method)

## Creating an interactor

In [7]:
interactor = Rays.Interactor(scene, params, affect_parameters_input!, affect_parameters_time!, get_render);

In [8]:
Rays.run!(interactor)

KeyError: KeyError: key 1 not found